# Equity Real World SVJD Tool Specifcation

Our aim here is to use the functionality in the **MoodysAnalytics.Calibration.Tool.RealWorldEquity** project to replicate the functionality in the excel **CalibrationTool_Best_Views_SVJD** used in production.  

### Excel tool example

See [EndSep2017_RW_SVJD_CalibrationTool_Best_Views_SVJD.xlsm](V:\Deliverables\StandardCalibrations\Calibration\EndSep2017\Real World\EndSep2017_RW_SVJD_CalibrationTool_Best_Views_SVJD.xlsm) for an example of the current process.

### See also - Constant Volatility

The layout of this tool will be similar to the **Equity Constant Volatility** tool. It is also a *multi-asset* tool that will create at the end of the process a fit for each of the individual assets that we define as part of the tool run.  

It will also take in as a Model input the result of the **Factor Calibration** performed by that tool. This will be the "combined" version of the results, and will reqiuire the **factorLoadingsArray** parameter.  

### Validation not included

We will need to replicate the *validation* step in the current excel tool. This will be done outside of the calibration tool method, and will be its own tool, with the approach TBD.  

It is likely we can use the same tool for both SVJD and constant volatility validation, albeit with lots of additional outputs that are required for SVJD only.  

## SVJD Calibration in Ghost

Like the constant volatility tool, there will be two stages of calibration, that come from separate tool run methods in the Ghost code base.  

We will also need to use a "sorter tool" for each asset, to extract the "per-asset" result from the final "combined" results.  

Unlike the constant volatilty tool, the first tool that we use is not a complete calibration on its own - it is an intermediate step on the way to the final calibration. This could lead to us considering creating a single tool wrapper around both methods, but I would lean toward separate Mercury tools so that these can be maintained separately.  

### SVJD Calibration in a nutshell

Boiled down to its core, the steps that we need to do in the SVJD calibration process is:

+ Calibrate the *Volatility* of the **First Equity Factor** as an SVJD model.
    + This is targeted to market *one month volatility* targets.
    + We start with the existing **Assets.EQ.Factors.Sigma.Const** model for all (6) factors, which are using a *Constant Volatility* model for their volatility.
    + We make the **First Equity Factor** an SVJD model, and recalibrate.
    + We need to calibrate to find the *Volatility* of this Factor at a term of **1m** when using and SVJD model
+ Calibrate the *Volatility* of **Each Equity Asset** using an SJVD model for the *specific volatility* of the asset.
    + The assets will take a *Systematic Volatility* as the product of the **Factor Calibration** and the **Factor Loadings**
    + We will take in the *Specific Volatility* for each asset - this is the result from the constant volatility calibration.
    + The **Factor Loadings** come from a constant volatility calibration, and remain unchanged.
    + The **Jump Parameters** of the **First Equity Factor** SVJD model are calibrated to market *Skew* and *Kurtosis* targets.
    + We then **calculate** the individual SVJD model calibration for each asset.
    
#### The "gotcha" here.

There are a lot of "in the margin" calculations that sit outside of the method outputs that produce the actual model calibration for the SVJD models for the **First Equity Factor** and **Each Equity Asset**.  

These do appear to all reside in the SVJD code base, so we should not need to port anything from the excel wrapper this time, touch wood.  

### TMT Diagram

We can start this time by showing the proposed TMT diagram for the entire SVJD calibration process.  

![](images/TMT_SVJD_3ToolSolution.svg)


So, as described in the steps above.  

We should now go a little deeper into the Ghost code, and see how we map into these tools.  

## Tool Method Mapping

The methods that we need to use from the **RealWorldEquity** tool in the Ghost project are:

+ The "SVJD Equity Factor Volatility Tool" is the Calculate method of:
    + **MoodysAnalytics.Calibration.DataContract.RealWorldEquity.RealWorldSVJD.FitConditionalFactorOne**
+ The "SVJD Equity Asset Volatility Tool" is the Calculate method of:
    + **MoodysAnalytics.Calibration.DataContract.RealWorldEquity.RealWorldSVJD.FitJumps**
+ We will also need a few methods from the above tools to calculate the final SVJD models
    + Which we will go into details of when we discuss mapping the output models.
    
### Additional Calculations

Word of warning - there are a *lot* of in-the-margin calculations that we need to perform to turn the tool result into the ESG calibration models that we need as the "final" answer.






### SVJD Equity Factor Volatility Tool Mapping

#### Inputs and Tool Settings

We need to map our model input data into the Inputs and ToolSettings classes for the calculate method.  

In [1]:
import pandas as pd
tablePCAMap = pd.read_json('data/SVJD_Factor_Input.json')
tablePCAMap = tablePCAMap.set_index(['Name'])
print("Table: Mapping SVJD Factor Volatility Data contract to Model Parameters.")
tablePCAMap[['Type','InputObj','AnchoredModel','ExampleModel','Parameter']]

Table: Mapping SVJD Factor Volatility Data contract to Model Parameters.


,Type,InputObj,AnchoredModel,ExampleModel,Parameter
Name,,,,,
factorLoadings,"double[,]",Inputs,Calibrated Factor Betas,Assets.EQ.PEA,FactorLoading.Table
oneMonthVolTargets,double[],Inputs,1m Volatility Targets Combined,Targets.Equity.Vol.1m,oneMonthVol.Table*
specificVol,double[],Inputs,Combinded Asset Constant Specific Volatility,Assets.EQ.PEA.Sigma.Const,CONST_Specific_Vols
factorVols,double[],Inputs,Starting Factor Volatility,Assets.EQ.Factors.Sigma.Const,FactorVols.All
weights,double[],Inputs,Tool Settings,Settings.Equity.SVJD.Factor,ConstVolInfo.Table['SVJD_OneMonthVol_Weight']
minVol,double,Inputs,Tool Settings,Settings.Equity.SVJD.Factor,SpecVol.MinimumValue
optimizerSettings,OptimiserSettings,ToolSettings,Tool Settings,Settings.Equity.SVJD.Factor,"""Optimizer.LM."" namespace parameters"
useAnalyticFitting,bool,ToolSettings,Tool Settings,Settings.Equity.SVJD.Factor,FactorCalibration.ApplyAnalyticsMethod


So points of interest here:

+ The **Assets.EQ.PEA** model is our combined result from the constant volatility tool.
    + We will need factor loadings and specific vols here.
+ We will require the original **Assets.EQ.Factors.Sigma.Const** model for the *Factor Vols*
    + We will be calibrating *Factor 1* here, but Factors 2-6 will remain the same.
+ The **Settings.Equity.SVJD.Factor** will contain a table of individual asset configs, in the same way as the constant vol tool
    + In fact, it is the same table that we used for constant vol.
+ Likewise, the optimiser settings are the same as in constantVol
    + There's no real difference in the "Settings" models at all at the moment, although these may diverge in the future.
+ The target here is the **Targets.Equity.Vol.1m** model - a vector of targets for *selected* assets
    + There are only targets for around 30 assets. 
    + Furthermore, the weights will reduce this to 6 assets
    
#### So, what does this tool do?

On completion of the calibration method of this tool, we will have fitted the *Volatility* of *Factor 1* to the target data. This is the **conditionalOneMonthFactorVol** output.   

There is still a long way to go to turn this into a useable Model calibration, but we will avoid doing any extra calculations in this tool - instead we will save all that for the **SVJD Equity Asset Tool** so we can do all of these calcs at the end in one go.  

#### Output Mapping

We can summarise what we need to map from the outputs class into our output models. This does not have much use on its own, so this is all for the benifit of the downstream **SVJD Equity Asset Tool**.

In [2]:
import pandas as pd
tablePCAMap = pd.read_json('data/SVJD_Factor_Output.json')
tablePCAMap = tablePCAMap.set_index(['OutputName'])
print("Table: Mapping SVJD Factor Volatility Data Outputs to Model Parameters.")
tablePCAMap[['OutType','ModelParameterName','ModelType','OutModel','ExampleOutModel','Description']]

Table: Mapping SVJD Factor Volatility Data Outputs to Model Parameters.


,OutType,ModelParameterName,ModelType,OutModel,ExampleOutModel,Description
OutputName,,,,,,
logChangeInSigma,double[],LogChangeInSigma.All,Vector,Output Intermediate FactorVol,Assets.EQ.Factors.PartialFit,NaN
sigma0,double[],Sigma0.All,Vector,Output Intermediate FactorVol,Assets.EQ.Factors.PartialFit,NaN
conditionalOneMonthFactorVol,double[],FactorVols.1m.All,Vector,Output Intermediate FactorVol,Assets.EQ.Factors.PartialFit,"F1.Sigma is calibrated, F2-F6 are as the input..."
error,double,Optimiser.ObjFn.Solution,Single,Output Intermediate FactorVol,Assets.EQ.Factors.PartialFit,NaN
stoppingMessage,string,Optimiser.StoppingMessage,Single,Output Intermediate FactorVol,Assets.EQ.Factors.PartialFit,NaN


### SVJD Equity Asset Tool Mapping

We can show how to map model input data into the Inputs and ToolSettings classes for this tool's calculate method.  

As well as taking in the result of the *Factor* tool, there is plenty overlap between the inputs of this tool and the previous tool.  

In [3]:
import pandas as pd
tablePCAMap = pd.read_json('data/SVJD_Asset_Input.json')
tablePCAMap = tablePCAMap.set_index(['Name'])
print("Table: Mapping SVJD Asset Volatility Data contract to Model Parameters.")
tablePCAMap[['Type','InputObj','AnchoredModel','ExampleModel','Parameter']]

Table: Mapping SVJD Asset Volatility Data contract to Model Parameters.


,Type,InputObj,AnchoredModel,ExampleModel,Parameter
Name,,,,,
skewWeights,double[],Inputs,Skew Targets Combined,Targets.Equity.Skew.1m,Skew.Value.Table
kurtosisWeights,double[],Inputs,Kurtosis Targets Combined,Targets.Equity.Kurtosis.1m,Kurtosis.Value.Table
skewTargets,double[],Inputs,Tool Settings,Settings.Equity.SVJD.Asset,SVJDInfo.Table['SVJD_Skew_Weight']
kurtosisTargets,double[],Inputs,Kurtosis Targets Combined,Targets.Equity.Kurtosis.1m,SVJDInfo.Table['SVJD_Kurtosis_Weight']
parametersFixed,ParametersFixedInOptimization,Inputs,Factor Volatility Intermediate Fit,Assets.EQ.Factors.PartialFit,"Translated ""Factor 1"" SVJD model"
unconditionalFactorVolsSkippingOne,double[],Inputs,Starting Factor Volatility,Assets.EQ.Factors.Sigma.Const,FactorVols.All (skipping F1)
factorLoadings,"double[,]",Inputs,Calibrated Factor Betas,Assets.EQ.PEA,FactorLoading.Table
unconditionalSpecificVol,double[],Inputs,Combined Asset Constant Specific Volatility,Assets.EQ.PEA.Sigma.Const,CONST_Specific_Vols
uncTotalVol**,NaN,NaN,Combined Asset Constant Specific Volatility,Assets.EQ.PEA.Sigma.Const,CONST_Total_Vols


Points of interest here:

+ The **Assets.EQ.Factors.PartialFit** model will be passed in here
    + As well as the **conditionalOneMonthFactorVol** output, we will need pretty much everything from this model for the additonal  calculations.
+ The **Assets.EQ.PEA** model is our combined result from the constant volatility tool.
    + We will need factor loadings, **Total** and specific vols in this tool.
+ We will require the original **Assets.EQ.Factors.Sigma.Const** model for the *Factor Vols*
    + We will be calibrating *Factor 1* here, but Factors 2-6 will remain the same.
+ The targets here are taken from arrays of target values for a selection of assets
    + Again there will be less assets with a target than the full set in the config table.
    + Weights are taken from the config table.
+ The parameters here with a double-star are not required for the tool method, but will be needed for the additional calculations.
+ The model **Settings.SVJD.Seeds** will be used to carry:
    + Seeds and bounds for parameters that need it
    + Default values for some parameters (where there is a Seed value only)  
    


##### Setting up the "ParametersFixedInOptimisation"

Before we can run the calculate method, we will need to fish around in the avaliable parameters to define the **parametersFixed** setting we need to pass to the optimiser.  

We can show this mapping in the table below.

In [4]:
import pandas as pd
tablePCAMap = pd.read_json('data/SVJD_ParamsFixed.json')
tablePCAMap = tablePCAMap.set_index(['ParametersFixedInOptimization'])
print("Table: Mapping the ParametersFixedInOptimization class to the Model inputs.")
tablePCAMap[['Value','Model Source']]

Table: Mapping the ParametersFixedInOptimization class to the Model inputs.


,Value,Model Source
ParametersFixedInOptimization,,
constVol,"=Math.Pow(FactorVols.All[1],2)",Assets.EQ.Factors.Sigma.Const
reversionSpeed,BE_SVJD_E_Var_RevRate_F1,Settings.SVJD.Seeds
currentVol,"=Math.Pow(conditionalOneMonthFactorVol[1],2)",Assets.EQ.Factors.PartialFit
correlation,BE_SVJD_E_Var_Correl_F1,Settings.SVJD.Seeds
volVar,BE_SVJD_E_Var_Vol_F1,Settings.SVJD.Seeds
jumpIntensity,BE_SVJD_E_Jump_Lambda_F1,Settings.SVJD.Seeds
deltaTime,=1/12,Calculated*


So here:

+ Some settings are coming in untouched from the **Settings.SVJD.Seeds** model
+ We need to calculate the starting vols.
    + (i'm using Math.Pow(x, 2) here so that we don't confuse volatility and variance with superscript notation.
    + The *constVol* setting is the Volatility of the original Factor 1 (about 14%) squared
    + The *currentVol* setting is the **one month** Factor 1 volatility squared, which we calibrater earlier
+ *deltaTime* is calculated here - it is always 1/12.

#### Tool Run and outputs

Finally, we are now able to run the tool with all the paramters now defined.  

Our outputs from this method will be as follows.

In [5]:
import pandas as pd
tablePCAMap = pd.read_json('data/SVJD_Asset_Output.json')
tablePCAMap = tablePCAMap.set_index(['OutputName'])
print("Table: Mapping SVJD Asset Volatility Data Outputs to Model Parameters.")
tablePCAMap[['OutType','ModelParameterName','ModelType','OutModel','ExampleOutModel','Description']]

Table: Mapping SVJD Asset Volatility Data Outputs to Model Parameters.


,OutType,ModelParameterName,ModelType,OutModel,ExampleOutModel,Description
OutputName,,,,,,
oneMonthSkewModel,double[],Skew.1m.Model,Vector,Output Combined AssetSVJD,Assets.EQ.PEA,NaN
oneMonthKurtosisModel,double[],Kurtosis.1m.Model.,Vector,Output Combined AssetSVJD,Assets.EQ.PEA,NaN
oneMonthAverageModelSkew,double,Skew.1m.Model.Average,Single,Output Combined AssetSVJD,Assets.EQ.PEA,NaN
oneMonthAverageModelKurtosis,double,Kurtosis.1m.Model.Average,Single,Output Combined AssetSVJD,Assets.EQ.PEA,NaN
unconditionalSpecificVol,double[],Asset.UnconditionalVol.All,Vector,Output Combined AssetSVJD,Assets.EQ.PEA,NaN
jumpMeanSolution,double,NaN,NaN,NaN,NaN,Pass to SVJD Calculations
jumpVolSolution,double,NaN,NaN,NaN,NaN,Pass to SVJD Calculations
error,double,Optimiser.ObjFn.Solution,Single,Output Combined AssetSVJD,Assets.EQ.PEA,NaN
stoppingMessage,string,Optimiser.StoppingMessage,Single,Output Combined AssetSVJD,Assets.EQ.PEA,NaN


Now for the tricky bit - converting these outputs into the correct **Model** outputs.  



In [6]:
### Calculating the Model outputs

#### Factor 1 SVJD Model

We now have enough information to return our SVJD model for the volatility of *Factor 1*. We can construct all of the parameters of this model as follows:  

In [7]:
tablePCAMap = pd.read_json('data/Factor1_SVJD.json')
tablePCAMap = tablePCAMap.set_index(['SVJD_Parameter_F1'])
print("Table: Mapping the SVJD Model for Factor 1 Vol to the Model inputs.")
tablePCAMap[['Value','Model Source']]

Table: Mapping the SVJD Model for Factor 1 Vol to the Model inputs.


,Value,Model Source
SVJD_Parameter_F1,,
CurrentVariance,"=Math.Pow(conditionalOneMonthFactorVol[1],2)",Assets.EQ.Factors.PartialFit
ReversionLevel,"f(Math.Pow(FactorVols.All[1],2),ReversionSpeed...",AdditionalOutputCalculations.CalculateSpecific...
ReversionSpeed,BE_SVJD_E_Var_RevRate_F1,Settings.SVJD.Seeds
VolVar,BE_SVJD_E_Var_Vol_F1,Settings.SVJD.Seeds
Correlation,BE_SVJD_E_Var_Correl_F1,Settings.SVJD.Seeds
JumpIntensity,BE_SVJD_E_Jump_Lambda_F1,Settings.SVJD.Seeds
JumpMean,jumpMeanSolution,Output Combined AssetSVJD*
JumpVol,jumpVolSolution,Output Combined AssetSVJD*


So the calculations for **ReversionLevel** will need some explaining.  

The function required to calculate this can be found in the *AdditionalOutputCalculations* class in the SVJD tool, called **CalculateSpecificReversionVariance** - I can quote its header here:

```csharp
public static double CalculateSpecificReversionVariance(double unconditionalSpecificVol,
            double reversionSpeed,
            double volVar, double correlation)
      {...}
```

And in this case, the function should be called as follows:

``` csharp

CalculateSpecificReversionVariance(Math.Pow(FactorVols.All[1],2),ReversionSpeed,VolVar,Correlation)

```

Where **,ReversionSpeed,VolVar,Correlation** are the parameters of the same model that we are taking from the seeds, and **FactorVols.All[1]** is the factor 1 volatility of the original Factor 1.  

Ok, so we now have one output model ready, the **un-adjusted** Factor 1 SVJD Model. Guess what happens next?  

But first, we need to calculate 3 flavors of Factor Volatility...

#### Factor Volatility

We can now define *three* different arrays of Factor Volatilities, for Factors 1-6. In each case, the Factors 2-6 remain the same, and the value of Factor 1's volatilty changes.  

What we can now define is:

1. The **Unconditional** Factor Vols - these are the static values as set in the model **Assets.EQ.Factors.Sigma.Const**
2. The **Conditional** Factor Vols, where Factor 1 Volatility is set to the *conditionalOneMonthFactorVol[1]* parameter solution from **Assets.EQ.Factors.PartialFit**
3. The **One Year Vol** Factor Vols, where Factor 1 Volatility can be calculated using the **vbDiscreteSVJDCumVol** function.

The **vbDiscreteSVJDCumVol** function can be found in the *StaticMethods* class in the SVJD tool, which I can quote here:

``` csharp
public static double vbDiscreteSVJDCumVol(double reversionLevel, double reversionSpeed, double currentVol,
            double volVar, double correlation, double time, double noTimesteps, double jumpIntensity, double jumpMean,
            double jumpVol)
        {...}
```

And for our **One Year Vol** value, we will need the  **un-adjusted** Factor 1 SVJD Model that we calculated previously - lets assume that exists as an object called "F1" so we can refer to its parameters below. The function will need to be parameterised as follows:

```
    vbDiscreteSVJDCumVol( F1.ReversionLevel, F1.ReversionSpeed, F1.CurrentVariance, F1.VolVar,
                            F1.Correlation, 1, 12, 0, 0, 0)
```

So here:

+ We use the unadjusted F1 SVJD model paramters
+ We switch off jumps (the values are all set to zero)
+ We evaluate at time = 1 using a monthly timestep (noTimesteps = 12)

For a shorthand to refer to these in functions later, we can name these factor vols as:

1. **FactorVols_Unc**
2. **FactorVols_1m**
3. **FactorVols_1Y**

Each of these will have a **FactorVols.All** vector parameter, using "term" to index the number of the factor. The first value, **FactorVols.All[1]**, will be the part of interest to us later. Remember that [2..6] are the same in each of the factor vols.  


##### Adjusted Factor 1 SVJD Volatility Model

We now have enough to calculate the first (!) of our required output models. We want to adjust the SVJD model used in the first factor model to adjust for the effect of **jumps** in the process.  

Let's refer to this as the **AF1** model, so that we have a shorthand to use for it.  

This is similar to the un-adjusted **F1** model we have seen earlier, but with a different calculation for the **CurrentVariance** and **ReversionLevel** parameters.  

To adjust these for jumps in the process, we need to find another helper function from the SVJD tool library. This one is found in the *SVJDDistributionCharacteristicsCalcs* class, and is called **JumpVariance**.

This is a one-liner, so I will quote it in full.  

``` csharp
public static double JumpVariance(double jumpIntensity, double jumpMean, double jumpVol)
        {
            return jumpIntensity*(jumpMean*jumpMean + jumpVol*jumpVol);
        }
        
```

We will parameterise this with the jump parameters in the **F1** model, which will be carried over to **AF1** - i.e. F1.JumpVol = AF1.JumpVol etc.

So our **AF1.JumpVariance** can be calculated as follows:

``` csharp
AF1.JumpVariance = JumpVariance(AF1.JumpIntensity, AF1.JumpMean, AF1.JumpVol)
```

We can now use this to calcuate the **AF1.ReversionLevel** parameter using again the **CalculateSpecificReversionVariance** function defined earlier.

``` csharp
AF1.ReversionLevel = Math.Max(0,CalculateSpecificReversionLevelOfVariance(
    FactorVols_1Y.FactorVols.All[1]*(FactorVols_1Y.FactorVols.All[1] - AF1.JumpVariance),
    AF1.ReversionSpeed,
    AF1.VolVar,
    AF1.Correlation))
```

The "Math.Max" step is a safety feature, you most likely will return the value of the function. The **AF1** parameters in the function can be safely swapped out for the **F1** equivalent - these are the same in both cases.  

Finally (for this model) the **AF1.CurrentVariance** is calculated as:

``` csharp

AF1.CurrentVariance = Math.Max(F1.CurrentVariance-AF1.JumpVariance, Math.Pow(SpecVol.MinimumValue,2)

```
Where **SpecVol.MinimumValue** is a global setting of this tool.  

So putting this all together, the parameters for the **AF1** model will be as follows:


In [8]:
tablePCAMap = pd.read_json('data/Factor1_SVJD_Adj.json')
tablePCAMap = tablePCAMap.set_index(['AF1 Parameters'])
print("Table: Mapping the Jump Adjusted SVJD Model for Factor 1 Vol (AF1) to the Model inputs.")
tablePCAMap[['Value','Model Source']]

Table: Mapping the Jump Adjusted SVJD Model for Factor 1 Vol (AF1) to the Model inputs.


,Value,Model Source
AF1 Parameters,,
CurrentVariance,AF1.CurrentVariance = Math.Max(F1.CurrentVaria...,Calculated*
ReversionLevel,"AF1.ReversionLevel = Math.Max(0,CalculateSpeci...",Calculated*
ReversionSpeed,F1.ReversionSpeed,F1
VolVar,F1.VolVar,F1
Correlation,F1.Correlation,F1
JumpIntensity,F1.JumpIntensity,F1
JumpMean,F1.JumpMean,F1
JumpVol,F1.JumpVol,F1


##### Per-Asset Conditional Constant Vol

We now want to evaluate the Equity Volatility model for each asset that we have loaded into the tool.  

For most assets, we want to convert the original constant vol model into an **SVJD** model. This is what we will do for each *equity* asset.  

For property assets - i.e. where *SVJDInfo.Table['IsProperty'] == True* - we want to return a **Constant Vol** model. As we have changed the factor model to have an SVJD model for its first factor, we will need to calculate and adjusted value for this - so it's not the same as the previous fit for the constantvol model.  

Note that in this tool we will evaluate the SVJD and ConstantVol model for each asset regardless of wether or not it is a property asset, and leave it up to configuration of the individual asset sorter tools to choose which equity model type to push.  

###### Constant Vol Calculations

Our ingredients for the adjusted Constant Vol calculation are:

+ The Factor Betas from the original constvol fit - *factorLoadings* from the *Calibrated Factor Betas* input.
+ The **Total** volatility from the original constvol fit - *uncTotalVol* from the *Combined Asset Constant Specific Volatility* input.
+ The **One Year Vol** Factor Vols - **FactorVols_1Y** from the calculations shown above in the *Factor Volatility* section.

To evaluate the paramters from the constant vol model, we need to calculate the following:

+ **CONST_Total_Vols** - are identical to the original parameter with the same name from the constVol tool.
+ **CONST_Systematic_Vols** - can be calculated as a function of the betas in the **FactorLoading.Table** we fit here, and the **FactorVols_1Y**
    + We can find this function in the **Constant Vol** code base as the method **CalculateSystematicVol**, in the class *GeneralEquityCorrelationCalcs* in the namespace *MoodysAnalytics.Calibration.Tool.RealWorldEquity.EquityBondCorrelation*
    + I'd consider this a straightforward enough a calculation that we could roll our own version of this within the tool wrapper, rather than have to bring the constant vol tool into play here.
+ **CONST_Specific_Vols** - can be calculated from Total and Systematic vols, bounded by the **SpecVol.MinimumValue** 
    + $ \sigma_{Specific} = \sqrt{\max{\{\sigma_{Minimium}^2, \sigma_{Total}^2 - \sigma_{Systematic}^2 \}}}$ 
    + Exactly how we did this in the ConstVol tool 
    
So, after this calculation we have 3 vectors of volatilites for each asset.  

Let's denote this model for each asset *i* as **C[i]**, so we can refer to this in the final output push table.  




##### SVJD Vol per-asset calculations

We now want to calculate an SVJD Volatility model for each asset we have loaded into the tool.  

Let's denote this **A[i]**, i.e. an **A**sset SJVD model for each *[i]* asset in the tool.   

Similar to how we have set up factors so far, **A[i].CurrentVariance** and **A[i].ReversionLevel** are calculated, and the other parameters can be populated from Seed values or from **F1**. Effectively, the Jumps are turned off for these assets - the values fror this come from the **Settings.SVJD.Seeds** model we load in.

We will however need to check if the asset is a **Property** asset, as we will use a different set of seeds when that case is true. Let's just hard code these, as its all 1's and 0's.  

Let's show the table of values first, then we can go into detail on the calculated values.  

In [9]:
tablePCAMap = pd.read_json('data/Asset_SVJD.json')
tablePCAMap = tablePCAMap.set_index(['A[i] Parameters'])
print("Table: Mapping the SVJD Model for Factor 1 Vol to the Model inputs.")
tablePCAMap[['Value-Equity','Value-Property','Model Source']]

Table: Mapping the SVJD Model for Factor 1 Vol to the Model inputs.


,Value-Equity,Value-Property,Model Source
A[i] Parameters,,,
CurrentVariance,"=Math.Pow(Sigma0.All[i],2)",Same as equity,Factor Volatility Intermediate Fit*
ReversionLevel,=CalculateSpecificReversionVariance(Math.Pow(C...,Same as equity,Calibrated Factor Betas*
ReversionSpeed,F1.ReversionSpeed,1,F1
VolVar,F1.VolVar,0,F1
Correlation,F1.Correlation,0,F1
JumpIntensity,BE_SVJD_E_Jump_Lambda,BE_SVJD_E_Jump_Lambda,Settings.SVJD.Seeds
JumpMean,BE_SVJD_E_Jump_Mean,BE_SVJD_E_Jump_Mean,Settings.SVJD.Seeds
JumpVol,BE_SVJD_E_Jump_Vol,BE_SVJD_E_Jump_Vol,Settings.SVJD.Seeds


So for the **A[i].CurrentVariance** parameter, we have a simple calculation - the value squared -  on the **Sigma0** parameter that we recieved way back from the *SVJD Equity Factor Volatility Tool*.  

For the **A[i].ReversionLevel**, we will need to use again the *CalculateSpecificReversionVariance* function, this time populated as:

``` csharp
A[i].ReversionLevel = CalculateSpecificReversionVariance(Math.Pow(CONST_Specific_Vols[i],2),
                        A[i].ReversionSpeed,
                        A[i].VolVar,
                        A[i].Correlation)
```

Using the **CONST_Specific_Vols** that we pull in under the *Calibrated Factor Betas* anchored model.  

#### Putting all this together - Final Push from the SVJD Equity Asset Tool

So what I have shown above as the "push" from the **SVJD Equity Asset Tool** is just the mapping of the tool output to some paramters, which on its own is not what we need. What we need to add alongside this "raw" fit are the calculated models we have created along the way, i.e. 

+ The **FactorVols_Unc**, **FactorVols_1m**, **FactorVols_1Y** factor vol arrays
+ The parameterised SVJD models for factors **F1** and **AF1**
+ The array of SVJD model parameters for each asset **A[i]**
+ The array of adjusted conditional ConstVol model parameters for each asset **C[i]**
+ A copy of the Factor Betas - this is the same as we take in.

So, adding all of these things together with the raw output, our revised output model push from this tool will be as follows...

The Output Model here is a **Assets.EQ.PEA** model, through the **Output Combined AssetSVJD** anchored model.  

In [10]:
tablePCAMap = pd.read_json('data/SVJD_Asset_FullOut.json')
tablePCAMap = tablePCAMap.set_index(['ModelParameterName'])
print("Table: Mapping SVJD Asset Volatility full calculated outputs to Model Parameters.")
tablePCAMap[['ModelType','OutputName','OutType','Source']]

Table: Mapping SVJD Asset Volatility full calculated outputs to Model Parameters.


,ModelType,OutputName,OutType,Source
ModelParameterName,,,,
LogChangeInSigma.All,Vector,LogChangeInSigma.All,double[],Factor Volatility Intermediate Fit
Sigma0.All,Vector,Sigma0.All,double[],Factor Volatility Intermediate Fit
FactorVols.1m.All,Vector,FactorVols.1m.All,double[],Factor Volatility Intermediate Fit
FactorLoading.Table,Table,FactorLoading.Table,"double[,]",Calibrated Factor Betas
Skew.1m.Model,Vector,oneMonthSkewModel,double[],From Raw output
Kurtosis.1m.Model.,Vector,oneMonthKurtosisModel,double[],From Raw output
Skew.1m.Model.Average,Single,oneMonthAverageModelSkew,double,From Raw output
Kurtosis.1m.Model.Average,Single,oneMonthAverageModelKurtosis,double,From Raw output
Optimiser.ObjFn.Solution,Single,error,double,From Raw output


#### Factor Model output

We can export the factor model calibration through an anchored output model called **Output SVJD FactorModel** as an *Assets.EQ.Factors.Sigma.SVJD* model. No need for a sorter tool here as this is a single model, and we have all of what we need.  

This will be mapped as follows, using some older parameter naming conventions. It is the **AF1** model that we push as the Factor calibraion.  

In [11]:
tablePCAMap = pd.read_json('data/SVJD_FactorVolModel.json')
tablePCAMap = tablePCAMap.set_index(['ModelParameterName'])
print("Table: Mapping SVJD Asset Volatility Factor Volatility outputs to Model Parameters.")
tablePCAMap[['ModelType','OutputName','OutType','Source']]

Table: Mapping SVJD Asset Volatility Factor Volatility outputs to Model Parameters.


,ModelType,OutputName,OutType,Source
ModelParameterName,,,,
BE_SVJD_E_Var_StartVal_F1,Single,AF1.CurrentVariance,double,AF1
BE_SVJD_E_Var_RevLevel_F1,Single,AF1.ReversionLevel,double,AF1
BE_SVJD_E_Var_RevRate_F1,Single,AF1.ReversionSpeed,double,AF1
BE_SVJD_E_Var_Vol_F1,Single,AF1.VolVar,double,AF1
BE_SVJD_E_Var_Correl_F1,Single,AF1.Correlation,double,AF1
BE_SVJD_E_Jump_Lambda_F1,Single,AF1.JumpIntensity,double,AF1
BE_SVJD_E_Jump_Mean_F1,Single,AF1.JumpMean,double,AF1
BE_SVJD_E_Jump_Vol_F1,Single,AF1.JumpVol,double,AF1


## Sorter Tool Stage

We now need to pass in the combined **Assets.EQ.PEA** model to a sorter tool to extract the SVJD *and* ConstVol model and push it as an individual model calibration.  

In [12]:
tablePCAMap = pd.read_json('data/Sorter_SVJD.json')
tablePCAMap = tablePCAMap.set_index(['ModelParameterName'])
print("Table: Mapping SVJD_Combined input to individual SVJD Model Parameters.")
tablePCAMap[['ModelType','OutputName','OutType','Source']]

Table: Mapping SVJD_Combined input to individual SVJD Model Parameters.


,ModelType,OutputName,OutType,Source
ModelParameterName,,,,
BE_SVJD_E_Var_StartVal,Single,Asset.SVJD.CurrentVariance[k],Vector[k],SVJD_Combined
BE_SVJD_E_Var_RevLevel,Single,Asset.SVJD.ReversionLevel[k],Vector[k],SVJD_Combined
BE_SVJD_E_Var_RevRate,Single,Asset.SVJD.ReversionSpeed[k],Vector[k],SVJD_Combined
BE_SVJD_E_Var_Vol,Single,Asset.SVJD.VolVar[k],Vector[k],SVJD_Combined
BE_SVJD_E_Var_Correl,Single,Asset.SVJD.Correlation[k],Vector[k],SVJD_Combined
BE_SVJD_E_Jump_Lambda,Single,Asset.SVJD.JumpIntensity[k],Vector[k],SVJD_Combined
BE_SVJD_E_Jump_Mean,Single,Asset.SVJD.JumpMean[k],Vector[k],SVJD_Combined
BE_SVJD_E_Jump_Vol,Single,Asset.SVJD.JumpVol[k],Vector[k],SVJD_Combined


Here we are reading off the value for each value for the current index *k*.

In [13]:
tablePCAMap = pd.read_json('data/Sorter_ConstVol.json')
tablePCAMap = tablePCAMap.set_index(['ModelParameterName'])
print("Table: Mapping SVJD_Combined input to individual ConstVol Model Parameters.")
tablePCAMap[['ModelType','OutputName','OutType','Source']]

Table: Mapping SVJD_Combined input to individual ConstVol Model Parameters.


,ModelType,OutputName,OutType,Source
ModelParameterName,,,,
BE_CONST_E_TotalVol,Single,Asset.ConstVol.CONST_Total_Vols[k],Vector[k],SVJD_Combined
BE_CONST_E_SystematicVol,Single,Asset.ConstVol.CONST_Systematic_Vols[k],Vector[k],SVJD_Combined
BE_CONST_E_SpecVol,Single,Asset.ConstVol.CONST_Specific_Vols[k],Vector[k],SVJD_Combined


And _finally_, the factor loading copy for the SVJD model. It will be identical to how the ConstVol sorter is processing this.  

In [14]:
tablePCAMap = pd.read_json('data/Sorter_Factors.json')
tablePCAMap = tablePCAMap.set_index(['ModelParameterName'])
print("Table: Mapping SVJD_Combined input to individual Factor Loading Model Parameters.")
tablePCAMap[['ModelType','OutputName','OutType','Source']]

Table: Mapping SVJD_Combined input to individual Factor Loading Model Parameters.


,ModelType,OutputName,OutType,Source
ModelParameterName,,,,
BE_E_Beta_f1,Single,"FactorLoading.Table[k,1]","Vector[k,i]",SVJD_Combined
BE_E_Beta_f2,Single,"FactorLoading.Table[k,2]","Vector[k,i]",SVJD_Combined
BE_E_Beta_f3,Single,"FactorLoading.Table[k,3]","Vector[k,i]",SVJD_Combined
BE_E_Beta_f4,Single,"FactorLoading.Table[k,4]","Vector[k,i]",SVJD_Combined
BE_E_Beta_f5,Single,"FactorLoading.Table[k,5]","Vector[k,i]",SVJD_Combined
BE_E_Beta_f6,Single,"FactorLoading.Table[k,6]","Vector[k,i]",SVJD_Combined
